# Обзор статьи Wasserstein GAN



Ссылка: https://arxiv.org/pdf/1701.07875.pdf

### #1 Введение:

В данной статье пойдет речь об обучении без учителя. Классический способ узнать распределение вероятностей -- это узнать плотность распределения. А именно, задается параметрическое семейство плотностей  $(P_\theta)_{\theta \in \mathbb{R}^d}$ и максимизируется логарифм правдоподобия:
$$\max_\limits{\theta \in \mathbb{R}^d}{\frac{1}{m}\sum\limits_{i=1}^{m}{log{P_\theta(x^{(i)})}}}$$

Если распределение реальных данных $\mathbb{P}_r$ допускает плотность, а $\mathbb{P}_\theta$ -- это распределение параметризованной плотности $P_\theta$, то, асимптотически, это сводится к минимизации дивергенции Кульбака-Лейблера 
$KL(\mathbb{P}_r || \mathbb{P}_\theta)$.
Но, для этого необходимо, чтобы существовала плотность, что не всегда выполнено, особенно, когда мы имеем дело с распределениями на низкоразмерных многообразияx. Тогда скорей всего дивергенция Кульбака-Лейблера не определена или просто бесконечна.

Вместо того, чтобы оценивать плотность, которая может не существовать, мы можем определить случайную величину $Z$ с конкретным распределением $p(z)$ и подать на вход параметризованной функции $g_\theta : \mathcal{Z} \rightarrow \mathcal{X}$ (обычно это нейронная сеть), которая напрямую генерирует объекты из распределения $\mathbb{P}_\theta$. Варьируя $\theta$ мы можем менять это распределение и делать его схожим на исходное распределение данных $\mathbb{P}_r$. Это может быть полезно по нескольким причинам. Во-первых, в отличие от плотностей, этот подход может представлять распределения на ограниченных низкоразмерных многообразиях. Во-вторых, возможность легко генерировать выборки полезнее, чем знание числового значения плотности.

Variational Auto-Encoders (VAEs) и Generative Adversarial Networks
(GANs) являются хорошо известными примерами этого подхода. Поскольку VAEs сосредоточены на приближении правдоподобия примеров, они разделяют ограничения стандартных моделей и должны возиться с дополнительными условиями шума. GAN предлагают гораздо большую гибкость в определении целевой функции. С другой стороны, обучение GAN очень нестабильное, что теоретически исследовано в других статьях.

В этой статье обращается внимание на различные способы измерения того, насколько близко находятся распределение модели и реальное распределение, или эквивалентно, на различные способы определения расстояния или дивергенции 
$\rho(\mathbb{P}_\theta, \mathbb{P}_r)$. Главное отличие между такими расстояниями заключается в их влиянии на сходимость последовательностей распределений вероятностей. Последовательность распределений 
$(\mathbb{P}_t)_{t \in N}$ сходится, если и только если существует распределение $\mathbb{P}_\infty$ такое, что 
$\rho(\mathbb{P}_t, \mathbb{P}_\infty)$ стремится к нулю, что зависит от того, как именно определяется расстояние $\rho$.

### #2 Разные расстояния:

Пусть $\mathcal{X}$ компактное метрическое множество и пусть $\Sigma$ множество всех борелевских подмножеств $\mathcal{X}$. Пусть $Prob(\mathcal{X})$ обозначает пространство вероятностных мер определенных на $\mathcal{X}$. Мы теперь можем определить расстояния между двумя распределениями 
$\mathbb{P}_r, \mathbb{P}_g \in Prob(\mathcal{X})$:

* Расстояние полной вариации (The *Total Variation* (TV) distance)

$$\delta(\mathbb{P}_r, \mathbb{P}_g) = \sup\limits_{A \in \Sigma}{|\mathbb{P}_r(A) - \mathbb{P}_g(A)|}$$

* Дивергенция Кульбака-Лейблера (The *Kullback-Leibler* (KL) divergence)

$$KL(\mathbb{P}_r || \mathbb{P}_g) = \int{log(\frac{P_r(x)}{P_g(x)})P_r(x)d\mu(x)}$$

* Дивергенция Дженсена-Шеннона (The *Jensen-Shannon* (JS) divergence)

$$JS(\mathbb{P}_r, \mathbb{P}_g) = KL(\mathbb{P}_r || \mathbb{P}_m) + KL(\mathbb{P}_g || \mathbb{P}_m)$$

где $\mathbb{P}_m = \frac{\mathbb{P}_r + \mathbb{P}_g}{2}$, эта дивергенция симметрична и всегда определена, так как мы можем выбрать $\mu = \mathbb{P}_m$

* The *Earth-Mover* (EM) distance или Wasserstein-1

$$W(\mathbb{P}_r, \mathbb{P}_g) = \inf\limits_{\gamma \in \Pi(\mathbb{P}_r, \mathbb{P}_g)}{\mathbb{E}_{(x, y) \text{~} \gamma}{[||x - y||]}}, \text{    (1)}$$

где $\Pi(\mathbb{P}_r, \mathbb{P}_g)$ обозначает множество всех совместных распределений $\gamma(x, y)$. Интуитивно понятно, что $\gamma(x, y)$ показывает, сколько «массы» должно быть перенесено из $х$ в $y$, чтобы преобразовать распределение $\mathbb{P}_r$ в распределение $\mathbb{P}_g$.

Далее приводится пример распределений, иллюстрирующий зачем нам нужно `Earth-Mover (EM) distance`. А именно, приводится последовательность распределений, которая сходится под `EM` расстоянием, но не сходится под всеми остальными. Также, этот пример показывает нам случай, когда мы можем узнать распределение вероятностей по низкоразмерному многообразию, выполнив градиентный спуск по `EM` расстоянию.

Далее в статье доказывается при каких условиях $W(\mathbb{P}_r, \mathbb{P}_g)$ является непрерывной функцией потерь.

#### Теорема #1
Пусть $\mathbb{P}_r$ распределение на $\mathcal{X}$. Пусть $Z$ случайная величина (например Гауссовская) над другим пространством $\mathcal{Z}$. Пусть $g : \mathcal{Z} \times \mathbb{R}^d \rightarrow \mathcal{X}$ функция, которую обозначим $g_\theta(z)$ с первым аргументом $z$, вторым $\theta$. Пусть $\mathbb{P}_\theta$ обозначает распределение $g_\theta(Z)$, тогда:

* Если $g$ непрерывна по $\theta \Rightarrow W(\mathbb{P}_r, \mathbb{P}_g)$ тоже непрерывна по $\theta$
* Если $g$ локально Липшицева и удовлетворяет (1), тогда $W(\mathbb{P}_r, \mathbb{P}_g)$ непрерывна везде, и дифференцируема почти всюду
* Утверждения 1-2 неверны для дивергенций Джейсена-Шеннона и Кульбака-Лейблера

Следующее следствие говорит нам, что обучение путем минимизации расстояния `EM` имеет смысл (по крайней мере, в теории) с нейронными сетями.

####  Следствие #1
Пусть $g_\theta$ нейронная сеть параметризованная $\theta$, и $p(z)$ априорное распределение над $z$, такое что $\mathbb{E}_{z\text{~}p(z)}[||z||] < \infty$ (например гауссовское, равномерное, и т.п.). Тогда предположение (1) выполнено и поэтому $W(\mathbb{P}_r, \mathbb{P}_\theta)$ непрерывна везде и дифференцируема почти всюду.

#### Теорема #2
Пусть $\mathbb{P}$ распределение на компакте $\mathcal{X}$ и $(\mathbb{P}_n)_{n \in \mathbb{N}}$ последовательность распределений на $\mathcal{X}$, тогда при устремлении $n \rightarrow \infty$,

* 1) $\delta(\mathbb{P}_n, \mathbb{P}) \rightarrow 0 
\iff JS(\mathbb{P}_n, \mathbb{P}) \rightarrow 0$
* 2) $W(\mathbb{P}_r, \mathbb{P}_g) \rightarrow 0 
\iff  \mathbb{P}_n \rightarrow \mathbb{P}$ (сходимость по распределению)
* 3) $KL(\mathbb{P}_n || \mathbb{P}) \rightarrow 0$ или $KL(\mathbb{P}_n || \mathbb{P}) \rightarrow 0$, подразумевая утверждение 1)
* 4) Утверждения в 1) подразумевают утверждения в 2)

Это подтверждает, что расстояния $KL$, $JS$ и $TV$ не годятся, как функция потерь, при изучении распределений, на низкоразмерных многообразиях. Однако расстояние $EM$ является разумным.

### #3 Wasserstein GAN:

Оптимизировать $\inf$ в определении $EM$ расстояния очень трудно. Но с другой стороны, двойственность Канторовича-Рубинштейна говорит нам, что

$$W(\mathbb{P}_r, \mathbb{P}_\theta) = 
\sup\limits_{||f||_L \leq 1}{\mathbb{E}_{x\text{~}\mathbb{P}_r}[f(x)] - \mathbb{E}_{x\text{~}\mathbb{P}_\theta}[f(x)]}$$

где супремум берется по всем 1-Липшецевым функциям $f\text{ : }\mathcal{X} \rightarrow \mathbb{R}$. Заметим, что если мы заменим $||f||_L \leq 1$ на $||f||_L \leq K$, тогда мы придем к $K W(\mathbb{P}_r, \mathbb{P}_g)$. Поэтому, имея семейство параметризованных функций $\{f_w\}_{w \in \mathcal{W}}$, которые все К-Липшецивы для некоторого K, наша задача сводится к следующей:

$$\max\limits_{w \in \mathcal{W}}{\mathbb{E}_{x\text{~}\mathbb{P}_r}[f_w(x)] - \mathbb{E}_{z\text{~}p(z)}[f_w(g_\theta(z))]}$$

#### Теорема #4
Пусть $\mathbb{P}_r$ какое-либо распределение. Пусть $\mathbb{P}_\theta$ распределение $g_\theta(Z)$, где $Z$ случайная величина с плотностью $p$ и $g_\theta$ функция удовлетворяющая (1). Тогда существует функция $f\text{ : }\mathcal{X}\rightarrow\mathbb{R}$ являющиеся решением: 

$$\max\limits_{||f||_L \leq 1}{\mathbb{E}_{x\text{~}\mathbb{P}_r}[f(x)] - \mathbb{E}_{x\text{~}\mathbb{P}_\theta}[f(x)]}$$

также имеем:

$$\nabla_\theta W(\mathbb{P}_r, \mathbb{P}_\theta) = 
-\mathbb{E}_{z\text{ ~ }p(z)}[\nabla_\theta f(g_\theta(z))]$$

Далее мы просто обучаем нейронную сеть, параметризованную своими весами лежащими в компакте $\mathcal{W}$. Для того чтобы веса $w$ лежали в компакте, самое
простое, что мы можем сделать, это ограничить их (скажем $\mathcal{W} = [-0.01; 0.01]^l$) после каждого апдейта градиента. Это очень грубые действия, и авторы статьи оставляют эту проблему на дальнейшее исследование. Более продвинутые техники можно найти в статье WGAN gradient penalty, в ней предлагают обеспечивать липшициевость функции за счет равенства нормы градиента единице почти всюду.

Рассмотрим алгоритм:

<img src="data/algorithm.png" width="800">

<img src="data/pic_1.png" width="800">

### #4 Эмпирические результаты:

Авторы статьи утверждают следующие два основных преимущества:

* функция потерь, которая коррелирует со сходимостью генератора и качеством образцов
* улучшение стабильности процесса оптимизации

Они сравнивают DCGAN и WGAN. Главной задачей было выучить распределение датасета LSUN-Bedrooms. 

<img src="data/pic_2.png" width="800">

Поскольку WGAN пытается обучить дискриминатор f (строки 2–8 в алгоритме) относительно хорошо до каждого обновления генератора (строка 10 в алгоритме 1), функция потерь в этой точке является оценкой $EM$ расстояния с точностью до константы, связанной с тем, как мы ограничиваем константу Липшица f. Наш первый эксперимент показывает, как эта оценка хорошо коррелирует с качеством сгенерированных образцов.

Одним из преимуществ WGAN является то, что он позволяет нам обучать дискриминатор до оптимального уровня. Когда дискриминатор обучен до конца, он просто дает генератору loss, на котором мы можем обучаться. Это говорит нам о том, что нам больше не нужно правильно балансировать мощность генератора и дискриминатора. Чем лучше дискриминатор, тем выше качество градиентов, которые мы используем для обучения генератора. Мы также установили, что WGAN намного более устойчив, чем GAN, при смене архитектуры генератора.

### #5 Вывод:

Мы представили алгоритм WGAN -- альтернатива традиционному обучению GAN. В этой новой модели мы показали, что можем улучшить стабильность обучения, предоставить содержательные кривые обучения, полезные для отладки и поиска гиперпараметров. Кроме того, мы показали, что соответствующая проблема оптимизации является обоснованной, и предоставили обширную теоретическую работу, подчеркивающую глубокие связи с другими расстояниями между распределениями.

Также в оригинальной статье гораздо больше теории, есть теория из смежных разделов и доказательства всех теорем.